In [1]:
import numpy as np
import os
import sys
import re 

In [2]:
import gaitnet
import classifier as cf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

In [3]:
SENSOR_DIR = "data/mobile_data/"
files = [(SENSOR_DIR + x) for x in os.listdir(SENSOR_DIR)]

In [4]:
name_filter = set(['ted','ausar','april'])
name_ind = {'ted':0, 'ausar':1, 'april':2}
ind_name = {0:'ted', 1:'ausar', 2:'april'}

In [5]:
test, label = cf.preprocess_csv(files[0], name_filter, name_ind)
print(test.shape, label)

In [6]:
features, labels = cf.generate_features(SENSOR_DIR, name_filter, name_ind)
print(features[0].shape, features.shape, labels.shape)

(15, 3) (89,) (89,)


In [7]:
features, labels = cf.reduce_feature_shape(features, labels, 13)
rand_ind = np.random.permutation(len(features))
features = torch.tensor(features).unsqueeze(2)
dataX, dataY = features[rand_ind], labels[rand_ind]
print(features[0].shape, features.shape, labels.shape)

torch.Size([13, 1, 3]) torch.Size([79, 13, 1, 3]) (79,)


In [8]:
features[0].shape

torch.Size([13, 1, 3])

In [9]:
gr = gaitnet.gaitAccelNet(n_people=3)
test = features[0].unsqueeze(0).float()
test_out = gr.forward(test)
# print(test.shape, test_out)

In [10]:
test2 = torch.cat([features[0].unsqueeze(0), features[1].unsqueeze(0)], 0).float()
test_out2 = gr.forward(test2)
# print(test_out2.shape, test_out2)

In [11]:
def train(data, model,optimizer,save_path,device='cpu',batch_size=2,epochs=1):
#     print("Starting the training process")
    features, labels = data
    datax = features.float()
    datay = torch.tensor(labels).long()
    
    model.train()
    
    model = model.to(device=device)
    loss_fcn = nn.CrossEntropyLoss()
    
    for e in range(epochs):
        for ind in range(len(datay)//batch_size):
            x = datax[ind*batch_size:(ind+1)*batch_size]
            y = datay[ind*batch_size:(ind+1)*batch_size]
            x, y = x.to(device), y.to(device)
            
            prediction = model(x)
#             print(prediction.shape, y.shape)
#             print(prediction, y)
            if (batch_size == 1):
                prediction = prediction.unsqueeze(0)
            loss = loss_fcn(prediction, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
#         if(e % 10 == 0):
#             print("at epoch {}".format(e))
        
    torch.save(model, save_path)
    return model

In [12]:
def validate(features, labels, trained_model):
    correct = 0
    trained_model.eval()
    for ind in range(len(labels)):
        test_ind = torch.tensor(labels[ind]) 
        ground_truth = labels[test_ind]
        tens = torch.tensor(features[test_ind]).unsqueeze(0).float()
        probs = F.softmax(trained_model.forward(tens), dim=0)
        prob_val, prob_ind = torch.max(probs, 0)
#         print(prob_ind, test_ind.long())
        if(prob_ind == test_ind.long()):
            correct += 1 
        
    return float(correct/len(labels)) * 100

In [13]:
def train_test_split(data, train_prop):
    dataX, dataY = data
    rand_ind = np.random.permutation(len(dataY))
    cutoff = int(len(dataY)*train_prop)
    
    model = gaitnet.gaitAccelNet(n_people=3)
    optimizer = optim.SGD(model.parameters(), lr = 1e-4, momentum=0.05)
    bmpath = "besh.pth"
    
    trainX, trainY = dataX[cutoff:], dataY[cutoff:]
    valX, valY = dataX[:cutoff], dataY[:cutoff]
    
    train([trainX, trainY], model, optimizer,bmpath, batch_size=1, epochs=50)
    
    acc = validate(valX, valY, model)
    return acc

In [14]:
model = gaitnet.gaitAccelNet(n_people=3)
optimizer = optim.SGD(model.parameters(), lr = 1e-4, momentum=0.05)
bmpath = "besh.pth"

print("Number of points = {}".format(len(dataY)))
tmod = train([dataX, dataY], model, optimizer, bmpath, batch_size=2, epochs=30)
validate(dataX[:len(dataY)//2], dataY[:len(dataY)//2], tmod)

Number of points = 79


c:\program files\python37\lib\site-packages\ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


0.0

In [18]:
print(tmod)

gaitAccelNet(
  (conv1): Conv2d(13, 52, kernel_size=(1, 1), stride=(1, 1))
  (conv2): Conv2d(52, 104, kernel_size=(1, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=104, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=3, bias=True)
)


In [15]:
train_test_split([dataX, dataY], 0.8)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


34.92063492063492

In [16]:
# import matplotlib.pyplot as plt
# accuracies = []
# for _ in range(50):
#     accuracies.append(train_test_split([dataX, dataY], 0.8))

# print("average accuracy = ", np.mean(np.array(accuracies)))    

# num_bins = 7
# n, bins, patches = plt.hist(accuracies, num_bins, facecolor='blue', alpha=0.5)
# plt.show()

In [17]:
loaded_model = gaitnet.gaitAccelNet(n_people=3)
loaded_model = torch.load(bmpath)